In [1]:
import geopandas as gpd
import pathlib
import random
from functools import reduce
from collections import defaultdict

import pandas as pd
import geopandas as gpd
import folium
import shapely
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import sklearn.cluster


In [2]:
import numpy as np

In [3]:
import folium

In [4]:
import pandas as pd


In [5]:
# 결측치 처리 된 인구 geojson 파일 불러옴
인구분포도 = gpd.read_file('output/suwon_500_people_true.geojson')
인구분포도

,gid,lbl,val,geometry
0,다사52a15b,1746.00,1746.0,"MULTIPOLYGON (((126.95885 37.23712, 126.95881 ..."
1,다사60a22b,1916.00,1916.0,"MULTIPOLYGON (((127.04866 37.30060, 127.04863 ..."
2,다사52a17b,4252.00,4252.0,"MULTIPOLYGON (((126.95872 37.25515, 126.95868 ..."
3,다사59b18b,8083.00,8083.0,"MULTIPOLYGON (((127.04324 37.26452, 127.04321 ..."
4,다사56a22a,6557.00,6557.0,"MULTIPOLYGON (((127.00356 37.29591, 127.00353 ..."
...,...,...,...,...
377,다사56b16a,185.00,185.0,"MULTIPOLYGON (((127.00955 37.24185, 127.00952 ..."
378,다사54a15a,6.00,6.0,"MULTIPOLYGON (((126.98142 37.23272, 126.98139 ..."
379,다사51b21b,17.00,17.0,"MULTIPOLYGON (((126.95282 37.29118, 126.95279 ..."
380,다사59a17b,8603.00,8603.0,"MULTIPOLYGON (((127.03765 37.25549, 127.03762 ..."


In [6]:
인구분포도.drop('geometry', axis=1).describe(include='all')

,gid,lbl,val
count,382,382,382.000000
unique,382,352,NaN
top,다사53b24a,6.00,NaN
freq,1,4,NaN
mean,NaN,NaN,3095.578534
std,NaN,NaN,2761.225087
min,NaN,NaN,6.000000
25%,NaN,NaN,488.500000
50%,NaN,NaN,2598.000000
75%,NaN,NaN,5035.500000


In [7]:
인구분포도.loc[인구분포도['val'].apply(pd.isna), 'val'] = 0
인구분포도

,gid,lbl,val,geometry
0,다사52a15b,1746.00,1746.0,"MULTIPOLYGON (((126.95885 37.23712, 126.95881 ..."
1,다사60a22b,1916.00,1916.0,"MULTIPOLYGON (((127.04866 37.30060, 127.04863 ..."
2,다사52a17b,4252.00,4252.0,"MULTIPOLYGON (((126.95872 37.25515, 126.95868 ..."
3,다사59b18b,8083.00,8083.0,"MULTIPOLYGON (((127.04324 37.26452, 127.04321 ..."
4,다사56a22a,6557.00,6557.0,"MULTIPOLYGON (((127.00356 37.29591, 127.00353 ..."
...,...,...,...,...
377,다사56b16a,185.00,185.0,"MULTIPOLYGON (((127.00955 37.24185, 127.00952 ..."
378,다사54a15a,6.00,6.0,"MULTIPOLYGON (((126.98142 37.23272, 126.98139 ..."
379,다사51b21b,17.00,17.0,"MULTIPOLYGON (((126.95282 37.29118, 126.95279 ..."
380,다사59a17b,8603.00,8603.0,"MULTIPOLYGON (((127.03765 37.25549, 127.03762 ..."


In [8]:
# Polygon의 중심점에 해당하는 좌표를 파생변수화하여, 위도-경도의 데이터로 뽑아와야 함

인구분포도['경도'] = 인구분포도['geometry'].centroid.x
인구분포도['위도'] = 인구분포도['geometry'].centroid.y
인구분포도

<ipython-input-8-1f38f2caf1b3>:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  인구분포도['경도'] = 인구분포도['geometry'].centroid.x
<ipython-input-8-1f38f2caf1b3>:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  인구분포도['위도'] = 인구분포도['geometry'].centroid.y


,gid,lbl,val,geometry,경도,위도
0,다사52a15b,1746.00,1746.0,"MULTIPOLYGON (((126.95885 37.23712, 126.95881 ...",126.961649,37.239390
1,다사60a22b,1916.00,1916.0,"MULTIPOLYGON (((127.04866 37.30060, 127.04863 ...",127.051467,37.302862
2,다사52a17b,4252.00,4252.0,"MULTIPOLYGON (((126.95872 37.25515, 126.95868 ...",126.961520,37.257418
3,다사59b18b,8083.00,8083.0,"MULTIPOLYGON (((127.04324 37.26452, 127.04321 ...",127.046041,37.266786
4,다사56a22a,6557.00,6557.0,"MULTIPOLYGON (((127.00356 37.29591, 127.00353 ...",127.006362,37.298176
...,...,...,...,...,...,...
377,다사56b16a,185.00,185.0,"MULTIPOLYGON (((127.00955 37.24185, 127.00952 ...",127.012352,37.244117
378,다사54a15a,6.00,6.0,"MULTIPOLYGON (((126.98142 37.23272, 126.98139 ...",126.984227,37.234984
379,다사51b21b,17.00,17.0,"MULTIPOLYGON (((126.95282 37.29118, 126.95279 ...",126.955622,37.293446
380,다사59a17b,8603.00,8603.0,"MULTIPOLYGON (((127.03765 37.25549, 127.03762 ...",127.040457,37.257750


In [9]:
# 인구분포도에서 val 값이 0인 지역은 설치할 이유가 없기에 제거
nobody = 인구분포도['val']==0.0
nobody
#mock_data[~mask].head() # ~를 포함하게 되면 mask의 값을 제외, ~을 제외하면 mask의 값을 포함입니다.



0      False
1      False
2      False
3      False
4      False
       ...  
377    False
378    False
379    False
380    False
381    False
Name: val, Length: 382, dtype: bool

In [10]:
인구분포도 = 인구분포도[~nobody]

In [11]:
인구분포도

,gid,lbl,val,geometry,경도,위도
0,다사52a15b,1746.00,1746.0,"MULTIPOLYGON (((126.95885 37.23712, 126.95881 ...",126.961649,37.239390
1,다사60a22b,1916.00,1916.0,"MULTIPOLYGON (((127.04866 37.30060, 127.04863 ...",127.051467,37.302862
2,다사52a17b,4252.00,4252.0,"MULTIPOLYGON (((126.95872 37.25515, 126.95868 ...",126.961520,37.257418
3,다사59b18b,8083.00,8083.0,"MULTIPOLYGON (((127.04324 37.26452, 127.04321 ...",127.046041,37.266786
4,다사56a22a,6557.00,6557.0,"MULTIPOLYGON (((127.00356 37.29591, 127.00353 ...",127.006362,37.298176
...,...,...,...,...,...,...
377,다사56b16a,185.00,185.0,"MULTIPOLYGON (((127.00955 37.24185, 127.00952 ...",127.012352,37.244117
378,다사54a15a,6.00,6.0,"MULTIPOLYGON (((126.98142 37.23272, 126.98139 ...",126.984227,37.234984
379,다사51b21b,17.00,17.0,"MULTIPOLYGON (((126.95282 37.29118, 126.95279 ...",126.955622,37.293446
380,다사59a17b,8603.00,8603.0,"MULTIPOLYGON (((127.03765 37.25549, 127.03762 ...",127.040457,37.257750


In [13]:
import folium

map = folium.Map(location=[37.2752207,127.0138527], tiles="cartodbpositron", zoom_start=13,
                 )
fmap=folium.Choropleth(geo_data = 인구분포도,
               data = 인구분포도,
               columns = ['gid', 'val'],
               fill_color = 'Blues',
               key_on='feature.properties.gid').add_to(map)
fmap.geojson.zoom_on_click = False

display(map)

map.save('output/인구밀도 시각화_500.html')

In [32]:
#하위 50% 지역을 추출함. (2598.000000)
구분포도_50 = 인구분포도.sort_values(by="val", ascending=False)
인구분포도_50 = 인구분포도_50[인구분포도_50["val"] <= 2598]
인구분포도_50

,gid,lbl,val,geometry,경도,위도
291,다사58a18b,2560.00,2560.0,"MULTIPOLYGON (((127.02632 37.26446, 127.02629 ...",127.029124,37.266720
206,다사61a21b,2539.00,2539.0,"MULTIPOLYGON (((127.05999 37.29163, 127.05997 ...",127.062802,37.293891
55,다사51b22a,2481.00,2481.0,"MULTIPOLYGON (((126.95279 37.29569, 126.95275 ...",126.955590,37.297953
81,다사52a16b,2478.00,2478.0,"MULTIPOLYGON (((126.95878 37.24614, 126.95875 ...",126.961584,37.248404
333,다사57b19b,2473.00,2473.0,"MULTIPOLYGON (((127.02062 37.27345, 127.02059 ...",127.023428,37.275711
...,...,...,...,...,...,...
202,다사54b19a,7.00,7.0,"MULTIPOLYGON (((126.98682 37.26880, 126.98679 ...",126.989621,37.271063
116,다사58a24a,6.00,6.0,"MULTIPOLYGON (((127.02601 37.31403, 127.02598 ...",127.028815,37.316295
54,다사57a26b,6.00,6.0,"MULTIPOLYGON (((127.01458 37.33652, 127.01455 ...",127.017386,37.338784
378,다사54a15a,6.00,6.0,"MULTIPOLYGON (((126.98142 37.23272, 126.98139 ...",126.984227,37.234984


In [33]:
인구분포도_50.to_csv("output/인구분포도_50_500.csv")

In [34]:
import folium

map = folium.Map(location=[37.2752207,127.0138527], tiles="cartodbpositron", zoom_start=13,
                 )
fmap=folium.Choropleth(geo_data = 인구분포도_50,
               data = 인구분포도_50,
               columns = ['gid', 'val'],
               fill_color = 'Blues',
               key_on='feature.properties.gid').add_to(map)
fmap.geojson.zoom_on_click = False

display(map)

map.save('output/인구밀도_50 시각화_500격자.html')

In [16]:

# 수원 지도 만들기
import folium
suwon_map = folium.Map(location=[37.50554861215234,126.77550612183495], zoom_start=13)


for name, lat, lng in zip(인구분포도.val, 인구분포도.위도, 인구분포도.경도):
    folium.CircleMarker([lat, lng],
                        radius=3,         # 원의 반지름
                        fill=True,
                        fill_color='purple',    # 원을 채우는 색
                        fill_opacity=0.7, # 투명도    
                        popup=name
    ).add_to(suwon_map)

display(suwon_map)
#suwon_map.save('map2.html')


# CCTV

In [18]:
data = gpd.read_file('output/suwon_only_500.geojson')
data

,gid,geometry
0,다사52a15b,"MULTIPOLYGON (((126.95885 37.23712, 126.95881 ..."
1,다사60a22b,"MULTIPOLYGON (((127.04866 37.30060, 127.04863 ..."
2,다사52a17b,"MULTIPOLYGON (((126.95872 37.25515, 126.95868 ..."
3,다사59b18b,"MULTIPOLYGON (((127.04324 37.26452, 127.04321 ..."
4,다사56a22a,"MULTIPOLYGON (((127.00356 37.29591, 127.00353 ..."
...,...,...
554,다사52b23b,"MULTIPOLYGON (((126.96397 37.30926, 126.96394 ..."
555,다사57b23a,"MULTIPOLYGON (((127.02042 37.30499, 127.02039 ..."
556,다사52b25a,"MULTIPOLYGON (((126.96388 37.32278, 126.96384 ..."
557,다사57a14a,"MULTIPOLYGON (((127.01530 37.22385, 127.01527 ..."


In [19]:
# 격자 데이터
data_gid = data[['gid', 'geometry']]

In [20]:
data_gid

,gid,geometry
0,다사52a15b,"MULTIPOLYGON (((126.95885 37.23712, 126.95881 ..."
1,다사60a22b,"MULTIPOLYGON (((127.04866 37.30060, 127.04863 ..."
2,다사52a17b,"MULTIPOLYGON (((126.95872 37.25515, 126.95868 ..."
3,다사59b18b,"MULTIPOLYGON (((127.04324 37.26452, 127.04321 ..."
4,다사56a22a,"MULTIPOLYGON (((127.00356 37.29591, 127.00353 ..."
...,...,...
554,다사52b23b,"MULTIPOLYGON (((126.96397 37.30926, 126.96394 ..."
555,다사57b23a,"MULTIPOLYGON (((127.02042 37.30499, 127.02039 ..."
556,다사52b25a,"MULTIPOLYGON (((126.96388 37.32278, 126.96384 ..."
557,다사57a14a,"MULTIPOLYGON (((127.01530 37.22385, 127.01527 ..."


In [21]:
import folium
import json
import pandas as pd
import numpy as np
import geopandas as gpd
import fiona
import os
import seaborn as sns
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, LineString, Point
from tqdm import tqdm

In [22]:
data3 = pd.read_csv("output/수원_cctv_정보.csv")
data3

,Unnamed: 0,관리기관명,소재지도로명주소,소재지지번주소,설치목적구분,카메라대수,카메라화소수,촬영방면정보,보관일수,설치연월,관리기관전화번호,위도,경도,데이터기준일자,제공기관코드,제공기관명
0,364,경기도지사,경기도 수원시 권선구 권광로 129,경기도 수원시 권선구 권선동 1015-6,생활방범,9,200.0,NaN,30.0,2000-01,031-230-1615,37.259429,127.030380,2017-04-19,B460004,대한적십자사
1,6277,경기도혈액원,경기도 수원시 권선구 권광로 129,경기도 수원시 권선구 권선동 1015-6번지,생활방범,6,200.0,NaN,30.0,2012-10,031-220-8500,37.259429,127.030380,2017-04-19,B460004,대한적십자사
2,6279,경기도혈액원,경기도 수원시 팔달구 덕영대로 923-1 새수원빌딩 4층,경기도 수원시 팔달구 매산로1가 57-104 새수원빌딩 4층,생활방범,2,200.0,NaN,30.0,2015-11,031-245-6518,37.266146,127.001686,2017-04-19,B460004,대한적십자사
3,7817,경기도 수원시,경기도 수원시 팔달구 창룡대로103번길 41,경기도 수원시 팔달구 매향동 121-136,생활방범,4,200.0,360도전방면,30.0,2017-05,031-228-3304,37.286650,127.022729,2020-07-31,3740000,경기도 수원시
4,7818,경기도 수원시,경기도 수원시 장안구 연무로13번길 13,경기도 수원시 장안구 연무동 239-22,생활방범,1,200.0,360도전방면,30.0,2017-05,031-228-3304,37.293289,127.028358,2020-07-31,3740000,경기도 수원시
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,48589,경기도 수원시,경기도 수원시 장안구 경수대로973번길 24,경기도 수원시 장안구 송죽동 453-31,생활방범,5,130.0,360도전방면,30.0,2011-01,031-228-3304,37.303401,127.003807,2020-07-31,3740000,경기도 수원시
216,48590,경기도 수원시,경기도 수원시 장안구 영화로13번길 14,경기도 수원시 장안구 영화동 428-14,생활방범,5,130.0,360도전방면,30.0,2011-01,031-228-3304,37.288197,127.009197,2020-07-31,3740000,경기도 수원시
217,48622,경기도 수원시,경기도 수원시 권선구 일월천로4번길 54,경기도 수원시 권선구 구운동 918-10,생활방범,4,200.0,360도전방면,30.0,2010-06,031-228-3304,37.282377,126.975143,2020-07-31,3740000,경기도 수원시
218,48653,경기도 수원시,경기도 수원시 권선구 매송고색로711번길 33,경기도 수원시 권선구 고색동 888-142,생활방범,5,200.0,360도전방면,30.0,2018-04,031-228-3304,37.252437,126.981037,2020-07-31,3740000,경기도 수원시


In [23]:
data3 = data3[["소재지지번주소","위도","경도"]]

In [24]:
data3

,소재지지번주소,위도,경도
0,경기도 수원시 권선구 권선동 1015-6,37.259429,127.030380
1,경기도 수원시 권선구 권선동 1015-6번지,37.259429,127.030380
2,경기도 수원시 팔달구 매산로1가 57-104 새수원빌딩 4층,37.266146,127.001686
3,경기도 수원시 팔달구 매향동 121-136,37.286650,127.022729
4,경기도 수원시 장안구 연무동 239-22,37.293289,127.028358
...,...,...,...
215,경기도 수원시 장안구 송죽동 453-31,37.303401,127.003807
216,경기도 수원시 장안구 영화동 428-14,37.288197,127.009197
217,경기도 수원시 권선구 구운동 918-10,37.282377,126.975143
218,경기도 수원시 권선구 고색동 888-142,37.252437,126.981037


In [25]:
data3['sub_cnt']=1
data3['경도'] = data3['경도'].astype(float)
data3['위도'] = data3['위도'].astype(float)
data3['point'] = data3.apply(lambda row : Point([row['경도'], row['위도']]), axis=1)
data3 = gpd.GeoDataFrame(data3, geometry='point')

<ipython-input-25-9f078b38371f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['sub_cnt']=1
<ipython-input-25-9f078b38371f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['경도'] = data3['경도'].astype(float)
<ipython-input-25-9f078b38371f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

In [27]:
gid_list = []
for i in tqdm(range(data3.shape[0])):
    point = data3['point'].iloc[i]
    if data_gid['geometry'].contains(point).sum() == 0:
        g1 = "Na"
    else :
        g = data_gid['gid'].loc[data_gid['geometry'].contains(point)]
        g1 = g.iloc[0]
    gid_list.append(g1)

100%|████████████████████████████████████████████████████████████████████████████████| 220/220 [00:02<00:00, 89.38it/s]


In [28]:
data3['gid'] = gid_list
data3['TL_cnt']=1
result3 = data3[['TL_cnt', 'gid']].groupby(['gid']).sum()
result3.reset_index(inplace=True)
data3 =result3


In [29]:
data3

,gid,TL_cnt
0,다사50a20b,1
1,다사50a21a,1
2,다사50b21a,1
3,다사51a18b,1
4,다사51b18b,3
...,...,...
125,다사62b16b,1
126,다사62b17a,3
127,다사62b17b,1
128,다사63a18b,1


In [30]:
data3.to_csv("output/has_cctv_500_true.csv")